In [3]:
import config
import pandas as pd
from pymongo import MongoClient

# Replace the uri string with your MongoDB deployment's connection string.
HOST = config.HOST
USER = config.USER
PASSWORD = config.PASSWORD
DATABASE_NAME = 'CP1_DB'
COLLECTION_NAME = 'population'
MONGO_URI = f"mongodb+srv://{USER}:{PASSWORD}@{HOST}/{DATABASE_NAME}?retryWrites=true&w=majority"

# 커넥션 접속 작업
client = MongoClient(MONGO_URI)
db = client[DATABASE_NAME] # Connection
collection = db[COLLECTION_NAME] # Creating table

# Converting cursor to the list of dictionaries
cursor = collection.find()
list_cur = list(cursor)
df = pd.DataFrame(list_cur)

# Printing the df to console
print(df.head())

# Close the connection to MongoDB when you're done.
client.close()

ConfigurationError: The DNS response does not contain an answer to the question: _mongodb._tcp.cluster0.1lslter.mongodb.net. IN SRV

## Transform data

In [ ]:
# 필요한 컬럼만 정리
df.drop(['_id','REG_DTTM'], axis=1,inplace=True)

In [ ]:
# 중복값 처리
print(df.shape)

df_processed = df.drop_duplicates()
print(df_processed.shape)

In [ ]:
','.join(df_processed.columns)

In [ ]:
tuple(df.iloc[0])

## 또다시, transform
- 구, 동, 지역 한국어로 바꿔야 함

In [ ]:
from TL import get_mongo_data

df = get_mongo_data()
df.head()

가벼운 전처리

In [ ]:
# 중복값, 필요없는 컬럼 제거
df.drop(['_id', 'REG_DTTM'], axis=1, inplace=True)
df.drop_duplicates(inplace=True)

이상치 제거하기

In [ ]:
# 1. 이상한 row 제거하기
# rows_drop = ['Seoul_Grand_Park']
df[df['AUTONOMOUS_DISTRICT'] == 'Seoul_Grand_Park'].index

In [ ]:
# 제거
print(df.shape)
df.drop(df[df['AUTONOMOUS_DISTRICT'] == 'Seoul_Grand_Park'].index, inplace = True)
df[df['AUTONOMOUS_DISTRICT'] == 'Seoul_Grand_Park']

In [ ]:
# 영어 동 이름 가공( 2동 같은 숫자 없애고 통합)
# 1/ Sinjeong4-dong 없애야함
# 2/ Deungchon-dong1 이런것도 없애야
df['ADMINISTRATIVE_DISTRICT'].sample(30)

In [ ]:
# 위의 두가지 요건을 만족시키기 위해 정규표현식 사용

df['ADMINISTRATIVE_DISTRICT'] = df['ADMINISTRATIVE_DISTRICT'].str.replace(r'\d-', '-').str.replace(r'dong\d', 'dong')

In [ ]:
# 제대로 변경되었는지 확인

df['ADMINISTRATIVE_DISTRICT'].sample(10)

In [ ]:
# 또 이상한 데이터 있나 확인
import pandas as pd
pd.set_option('display.max_rows',80)
df['ADMINISTRATIVE_DISTRICT'].value_counts()

In [ ]:
df[df['ADMINISTRATIVE_DISTRICT']=='Myeong-dong']

## 네이버 블로그 데이터 추출

In [ ]:
import requests
import config as c

keyword = '홍대 맛집'  # 필요할 경우 예외처리 추가
display = 100  # 한 번에 표시할 검색 결과 개수 (네이버 기본값: 10, 최댓값: 100)

API = f'https://openapi.naver.com/v1/search/blog.json?query={keyword}&display={display}'
Client_ID = '_VDq_u85WMw1RIiLChsi'
Client_Secret = 'HNoCcEOhg9'

# API 데이터 추출
raw_json = requests.get(API,
                        params={'query': keyword, 'display':display},
                        headers={'X-Naver-Client-Id':Client_ID,'X-Naver-Client-Secret':Client_Secret}
                        )

In [ ]:
import json
status = raw_json.status_code

if status==200:
    contents = json.loads(raw_json.text)

    print('정상조회되었습니다. 100건 중 샘플 3건의 게시물을 불러옵니다.')
    print(contents['items'][0:4])

In [ ]:
# Parsing
data = contents['items']

# tokenizing 할 텍스트들
titles = [idx['title'] for idx in data]
descriptions = [idx['description'] for idx in data]

In [ ]:
# 텍스트에서 전처리 함수 추가
import re
import random

def processing(sentence):
    # <b> 태그 제거 : 그런데 이런식으로 제거하는게 좀 비효율적으로 보인다. 좋은 방법이 없을까?
    sentence = sentence.replace('<b>','').replace('</b>','').replace('&apos;','')

    # 문자만 추출
    pat = re.compile('[^\w ]')
    sent = re.sub(pat,'', sentence)
    return sent

test = random.randint(0,99)
print(descriptions[test])
print(processing(descriptions[test]))

In [ ]:
from kolnpy.tag import Mecap

